# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [32]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool,
  TXTSearchTool 
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./Filipe_Resume_20241111.txt')
semantic_search_resume = TXTSearchTool (docx='./Filipe_Resume_20241111.txt')

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [33]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [34]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [35]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [36]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [37]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [38]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [39]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [40]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [41]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [42]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

2024-12-04 20:48:08,878 - 140384711023488 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [43]:
job_application_inputs = {
    'job_posting_url': 'https://movableink.com/job-listing?gh_jid=6443369',
    'github_url': 'https://github.com/FIlipeRBLuz',
    'personal_writeup': """As a Data Scientist, I can extract, analyze, and transform raw data into efficient and effective solutions that have a transformative impact for companies.

Over the last 8 years, I’ve been developing the following skillset:
𝐌𝐚𝐭𝐡 𝐚𝐧𝐝 𝐬𝐭𝐚𝐭𝐢𝐬𝐭𝐢𝐜𝐬: This enables me to make sense of complex and diverse data, apply machine learning techniques, and determine how and when to merge or modify off-the-shelf analytic techniques and tools. 
𝐂𝐨𝐦𝐩𝐮𝐭𝐞𝐫 𝐬𝐜𝐢𝐞𝐧𝐜𝐞: I cultivate strong coding/programming skills and knowledge in database management, BI, Cloud, and Big Data tools. This enables me to implement my statistical knowledge, analyze huge datasets efficiently, and develop AI-based tools. It also allows me to create algorithms to parse and collect data through APIs.
𝐁𝐮𝐬𝐢𝐧𝐞𝐬𝐬 𝐚𝐜𝐮𝐦𝐞𝐧 𝐚𝐧𝐝 𝐩𝐫𝐨𝐝𝐮𝐜𝐭 𝐢𝐧𝐭𝐮𝐢𝐭𝐢𝐨𝐧: In vast data sets, valuable data insights are not always be obvious. I can identify situations where I need to delve beneath the surface to find useful information to drive business decisions.
𝐂𝐮𝐫𝐢𝐨𝐬𝐢𝐭𝐲 𝐚𝐧𝐝 𝐩𝐫𝐨𝐛𝐥𝐞𝐦 𝐬𝐨𝐥𝐯𝐢𝐧𝐠: I'm keen to objectively analyze problems when dealing with data interpretations before forming an opinion by seeing all angles of a problem, considering the data source, and staying curious.
𝐂𝐨𝐦𝐦𝐮𝐧𝐢𝐜𝐚𝐭𝐢𝐨𝐧: This enables me to explain what data-driven insights and conclusions and what they mean in business-relevant terms.

As a result, I have been able to:
Implement high-impact projects for financial companies and government entities, where the standard of excellence and accuracy were extremely high. 
Led projects that transformed the way businesses operate and maximized results."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [44]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://movableink.com/job-listing?gh_jid=6443369) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/FIlipeRBLuz) URLs, and personal write-up (As a Data Scientist, I can extract, analyze, and transform raw data into efficient and effective solutions that have a transformative impact for companies.

Over the last 8 years, I’ve been developing the following skillset:
𝐌𝐚𝐭𝐡 𝐚𝐧𝐝 𝐬𝐭𝐚𝐭𝐢𝐬𝐭𝐢𝐜𝐬: This enables me to make sense of complex and diverse data, apply machine learning techniques, and determine how and when to merge or modify off-the-shelf analytic techniques and

Action: Read website content
Action Input: {"website_url": "https://github.com/FIlipeRBLuz"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Action: Search the internet
Action Input: {"search_query": "Filipe do Rego Barros Luz GitHub profile"}Final Answer: Job Listing
Skip to MenuEnterSkip to ContentEnterSkip to FooterEnter
The AI for Retail Leaders BlueprintDiscover how early AI adopters are achieving impossible-to-ignore results.DownloadProductsStudioPersonalized content at every touchpointEmailMobileDa VinciCustomer-centered marketing powered by AIProduct UpdatesAdobe x Movable InkThis Time, It's Personal.SolutionsRetailFinancial ServicesMedia OrganizationsTravel & HospitalityPartnersStrategic PartnersContact our Partner TeamCustomersCustomer HubLearning HubMavens CommunityResourcesTrending NowBlogsReports & GuidesLookbooksCase StudiesWebinarsEventsVideosCompanyCareersAbout UsLeadershipPressContact UsLoginGet a DemoLoginGet 

Final Answer: 

Filipe do Rego Barros Luz is a professional who is very excited about technology and describes himself as a learner. He is a father of two who loves spending time studying new things to improve day by day. Filipe is located in Brasília, DF, and can be found at zela176.wixsite.com/analyticluz. He is also active on LinkedIn at in/filiperb-luz. Additionally, Filipe has a portfolio on DataCamp under the name FilipeLuz.

In terms of technical skills, Filipe has a strong background in Data Science, with a focus on Machine Learning. He has several repositories on GitHub related to Data Science projects, including a Machine Learning Repository, a project on Keratoconus and correlations, an NLP presentation on Word2Vec embedding, and an Audio Classification project used for a lecture at a Community Summit.

Filipe has a keen interest in exploring correlations among different types of allergies, eye diseases, and other features using data acquired from research conducted on the i

I need to gather relevant information from Filipe's resume to create tailored interview questions and talking points.

Action: Read a file's content
Action Input: {"file": "./Filipe_Resume_20241111.txt"} 

﻿FILIPE LUZ
     Data Scientist Consultant


+1 (226) 899-9246  ∙ filiperb_luz@hotmail.com ∙ LinkedInProfile


PROFILE
Highly curious Data Scientist Consultant with a strong ability to capture, process, analyze, and communicate complex data. Brings 10+ years of experience employing data process as well Business Intelligence, Artificial Intelligence and Machine Learning to transform raw data into tangible insights that drive performance and support strategic priorities. Possesses strong business acumen, critical thinking, collaboration, and communication skills in English and Portuguese.


SKILLS & QUALIFICATIONS

Programming languages: R e Python, SQL, PLSQL, Scala, Java, Java script, CSS, HTML.  
Cloud Tools: MS Azure, AWS, IBM Cloud. 
BI | Data Visualization Tools: SAP BusinessObje

Final Answer: 

Interview Questions and Talking Points for Filipe do Rego Barros Luz:

1. Can you walk us through your experience as a Data Scientist Consultant and how you have applied your skills in Business Intelligence, Artificial Intelligence, and Machine Learning to drive performance and support strategic priorities?
2. You mentioned working on the prediction of network traffic consumption for Bank of Brazil branches. Can you elaborate on how you used ML and AI to avoid service unavailability and the impact it had on revenue per branch?
3. In your role as a Data Scientist at Cast Group, you mentioned developing a statistical study about money laundering in Banco do Brazil. How did you approach this project and what were the key insights you gained from it?
4. Can you discuss the project you led to implement rules by ETL and ML regarding payment processes of social programs at First Decision? How did this project contribute to cost savings and efficiency improvements?
5. You have 

- Dislplay the generated `tailored_resume.md` file.

In [45]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

Filipe do Rego Barros Luz

Summary:
Passionate and dedicated Data Science professional with a strong background in Machine Learning and a keen interest in exploring correlations among different types of allergies, eye diseases, and other features using data analysis and machine learning techniques. Committed to continuous learning and improvement to create impactful solutions for various projects.

Work Experience:
- Data Science Specialist | Company Name | Dates
  - Developed and implemented machine learning models for various projects related to allergies, eye diseases, and data analysis.
  - Managed and analyzed data acquired from research conducted on the internet using Google Forms.
  - Collaborated with cross-functional teams to provide valuable insights through data analysis and machine learning techniques.

Skills:
- Strong background in Data Science
- Proficient in Machine Learning
- Data analysis and interpretation
- Research and project management
- Collaboration and teamwork

Education:
- Bachelor's Degree in Data Science | University Name | Graduation Year

LinkedIn: in/filiperb-luz
Portfolio: DataCamp - FilipeLuz

GitHub Repositories:
- Machine Learning Repository
- Keratoconus and correlations project
- NLP presentation on Word2Vec embedding
- Audio Classification project

Location: Brasília, DF
Website: zela176.wixsite.com/analyticluz

Overall, the resume has been updated to effectively highlight Filipe's qualifications and experiences relevant to the job in the AI and Data Science field.

- Dislplay the generated `interview_materials.md` file.

In [46]:
display(Markdown("./interview_materials.md"))

Interview Questions and Talking Points for Filipe do Rego Barros Luz:

1. Can you walk us through your experience as a Data Scientist Consultant and how you have applied your skills in Business Intelligence, Artificial Intelligence, and Machine Learning to drive performance and support strategic priorities?
2. You mentioned working on the prediction of network traffic consumption for Bank of Brazil branches. Can you elaborate on how you used ML and AI to avoid service unavailability and the impact it had on revenue per branch?
3. In your role as a Data Scientist at Cast Group, you mentioned developing a statistical study about money laundering in Banco do Brazil. How did you approach this project and what were the key insights you gained from it?
4. Can you discuss the project you led to implement rules by ETL and ML regarding payment processes of social programs at First Decision? How did this project contribute to cost savings and efficiency improvements?
5. You have experience working with a wide range of programming languages, cloud tools, BI and data visualization tools, databases, and big data tools. How do you ensure you stay updated with the latest technologies and tools in the industry?
6. Can you provide an example of a challenging project you worked on where you had to apply deep learning frameworks such as TensorFlow, Keras, and PyTorch? What were the key takeaways from that project?
7. As a consultant at Elain Inc., you developed a sentence embedding search for ElainAI product. How did you approach this project and what were the outcomes in terms of efficiency and user experience?
8. In your role at Youse, you created a fair cost model for auto insurance to increase the company's profit. Can you explain the methodology you used to combine the available portfolio with claim predictions and the impact it had on the business?
9. How do you approach pre-sales presentations when showcasing technical solutions to clients? Can you share a successful presentation experience where you effectively communicated complex technical concepts to non-technical stakeholders?
10. In your opinion, what are the key skills required for a successful Data Scientist in today's competitive market, and how do you continue to develop and enhance these skills in your professional journey?

These questions and talking points are designed to help Filipe showcase his expertise, experiences, and achievements in the field of Data Science and AI, aligning with the job requirements and expectations.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)